# Resnet 56 - CIFAR10

## Comparing multiple executions

* How stable is the ranking of discovered networks?
 
 
Let's run 4 instances of AMC (ResNet56, DDPG) and then review the results.

## Multiple experiment runs

Below we plot the execution of the same experiment several times.<br>
We configure AMC as follows:
- Architecture: Resnet56 (CIFAR10)
- Action range: 5% - 100% retain rate
- Target density: 50%
- Don't fine-tune a discovered network before providing a reward.

The command-line is provided below:
    
    time python3 multi-run.py /home/young/liuyixin/CAMC_disllter/experiments/resnet56-ddpg-private amc.py  --arch=resnet56_cifar /home/dataset/cifar --state_dict=/home/young/liuyixin/CAMC_disllter/checkpoints/pytorch_resnet_cifar10/cifar10-resnet56-f5939a66.pth --amc-protocol=mac-constrained --amc-action-range 0.05 1.0 --amc-target-density=0.5 -p=50 --etes=0.075 --amc-ft-epochs=0 --amc-prune-pattern=channels --amc-prune-method=fm-reconstruction --amc-agent-algo=DDPG --amc-cfg=auto_compression_channels.yaml --amc-rllib=hanlab -j=1
    
Each colored line represents one experiment execution instance. We plot the search-Top1 score of discovered networks as the RL-based AMC system learns to find better compressed networks. You might be impressed by:
* The variability in behavior, which is typical for RL algorithms.
 

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib 
import csv
from matplotlib.ticker import FuncFormatter
import ipywidgets as widgets
from ipywidgets import interactive, interact, Layout
import matplotlib.pylab as pylab
import matplotlib.animation as animation
from matplotlib import animation, rc

from auto_compression_jupyter import *

#plt.style.use('seaborn') # pretty matplotlib plots

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 7),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'xx-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

    
# Widen the cells to get entire rows in the screen.
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

EXPERIMENTS_DIR = "/home/young/liuyixin/CAMC_disllter/experiments"

In [2]:
fpath = os.path.join(EXPERIMENTS_DIR, "resnet20-ddpg-private", "2020.09.15-193705")
df_list = load_experiment_instances(fpath)